## Todo: Parse Solution is our homework

In [1]:
from functools import lru_cache

def memo(f):
    memo.solution = {}
    def _wrap(*args):
        return f(*args)
    return _wrap

@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    
    if string1 == string2: return 0
    if len(string1) == 0: 
        memo.solution[(string1, string2)] = 'ADD_ALL {}'.format(string2)
        return len(string2)
    if len(string2) == 0: 
        memo.solution[(string1, string2)] = 'DEL_ALL {}'.format(string1)
        return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  
        # string 1 delete tail
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  
        # string 1 add tail of string2
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    memo.solution[(string1, string2)] = operation 
    
    return min_distance

def parse_action(str1, str2, res=[]):
    if str1 == str2:
        return []

    action = memo.solution[(str1, str2)]
    if "ALL" in action:
        pass
    elif "ADD" in action:
        parse_action(str1, str2[:-1], res)
    elif "DEL" in action:
        parse_action(str1[:-1], str2[:], res)
    else:
        parse_action(str1[:-1], str2[:-1], res)       
   
    res.append("'%s' -> '%s' : '%s'" % (str1, str2, action))
    return res

@memo
def parse_solution(str1, str2):
    print("min distance is", edit_distance(str1, str2))
    print("\n".join(parse_action(str1, str2, [])))

In [2]:
parse_solution('','ABACD')

min distance is 5
'' -> 'ABACD' : 'ADD_ALL ABACD'


In [3]:
parse_solution('ABCDECG','ABCCEF')

min distance is 3
'ABCD' -> 'ABCC' : 'SUB D => C'
'ABCDE' -> 'ABCCE' : ''
'ABCDEC' -> 'ABCCEF' : 'SUB C => F'
'ABCDECG' -> 'ABCCEF' : 'DEL G'


In [4]:
parse_solution('ABCD','ABACD')

min distance is 1
'AB' -> 'ABA' : 'ADD A'
'ABC' -> 'ABAC' : ''
'ABCD' -> 'ABACD' : ''


# Problem Case 3: Pinyin Auto Correction Problem

In [5]:
chinese_dataset = 'article_9k.txt'

In [6]:
CHINESE_CHARATERS = open(chinese_dataset).read()

In [7]:
CHINESE_CHARATERS[:40]

'此外自本周6月12日起除小米手机6等15款机型外其余机型已暂停更新发布含开发版体'

In [8]:
import pinyin

In [9]:
pinyin.get('你好，中国',format='strip',delimiter=' ')

'ni hao ， zhong guo'

In [10]:
def chinese_to_pinyin(character):
    return pinyin.get(character, format='strip', delimiter=' ')

In [11]:
CHINESE_CHARATERS_COPYS = chinese_to_pinyin(CHINESE_CHARATERS)

In [12]:
len(CHINESE_CHARATERS_COPYS)

129433034

In [13]:
import re

In [14]:
def tokens(text):
    "List all the pinyin characters"
    return re.findall('[a-z]+',text.lower())

In [15]:
CHINESE_CHARATERS_COPYS[:100]

'ci wai zi ben zhou 6 yue 1 2 ri qi chu xiao mi shou ji 6 deng 1 5 kuan ji xing wai qi yu ji xing yi '

In [16]:
tokens(CHINESE_CHARATERS_COPYS[:100])

['ci',
 'wai',
 'zi',
 'ben',
 'zhou',
 'yue',
 'ri',
 'qi',
 'chu',
 'xiao',
 'mi',
 'shou',
 'ji',
 'deng',
 'kuan',
 'ji',
 'xing',
 'wai',
 'qi',
 'yu',
 'ji',
 'xing',
 'yi']

In [17]:
from collections import Counter, defaultdict

In [18]:
PINYIN_COUNT = Counter(tokens(CHINESE_CHARATERS_COPYS))

In [19]:
def correct(word):
    'Find the most possible pinyin based on edit distance'
    # Prefer edit distance 0, then 1, then 2; otherwist default to word itself
    candidates = (known(edits0(word)) or
                  known(edits1(word)) or
                  known(edits2(word)) or
                  [word])
    return max(candidates,key=PINYIN_COUNT.get)

def correct(word):
    'Find the most possible pinyin based on edit distance'
    # Prefer edit distance 0, then 1, then 2; otherwist default to word itself
    candidates = (known(edits0(word)) or
                  known(edits1(word)) or
                  [word])
    return max(candidates,key=PINYIN_COUNT.get)

In [20]:
def known(words):
    'Return the pinyin in our data'
    return {w for w in words if w in PINYIN_COUNT}

def edits0(word):
    'Return all strings that are zero edits away from word (i.e., just word itself).'
    return {word}

def edits2(word):
    'Return all strings that are two edits away from this pinyin.'
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [21]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'

def splits(word):
    'Return a list of all possible (first, rest) pairs that comprise pinyin.'
    return [(word[:i], word[i:])
           for i in range(len(word)+1)]

def edits1(word):
    'Return all strings that are one edit away from this pinyin.'
    pairs = splits(word)
    deletes = [a+b[1:] for (a,b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a,b) in pairs if len(b) > 1]
    replaces = [a+c+b[1:] for (a,b) in pairs for c in alphabet if b]
    inserts = [a+c+b for (a,b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

In [22]:
splits('pinyin')

[('', 'pinyin'),
 ('p', 'inyin'),
 ('pi', 'nyin'),
 ('pin', 'yin'),
 ('piny', 'in'),
 ('pinyi', 'n'),
 ('pinyin', '')]

In [23]:
print(edits0('pinyin'))

{'pinyin'}


In [24]:
print(edits1('pinyin'))

{'pinyain', 'ypinyin', 'plnyin', 'piunyin', 'pinyinu', 'pijnyin', 'gpinyin', 'pinyinq', 'pxnyin', 'pinyinj', 'oinyin', 'pipnyin', 'pinyip', 'pinwin', 'pinyn', 'pinyzin', 'pinoin', 'pinpyin', 'pinyiny', 'pfinyin', 'pinyxn', 'pixnyin', 'pinyvn', 'pinyiq', 'pidnyin', 'pjinyin', 'pindyin', 'pinyini', 'piynin', 'pinyix', 'rinyin', 'pinyii', 'pieyin', 'pifnyin', 'piniyin', 'pinkyin', 'ginyin', 'pbnyin', 'pimyin', 'pinyio', 'piryin', 'xpinyin', 'piknyin', 'pinyina', 'pioyin', 'yinyin', 'pikyin', 'binyin', 'ponyin', 'pinyiy', 'upinyin', 'pginyin', 'pinypin', 'pinyinn', 'pcnyin', 'ptinyin', 'vinyin', 'pinybn', 'pianyin', 'pvinyin', 'piniyn', 'pincyin', 'pinyint', 'pinkin', 'pkinyin', 'panyin', 'qinyin', 'pindin', 'pinnin', 'pinyjn', 'ipinyin', 'pinjyin', 'pinyicn', 'ainyin', 'pimnyin', 'pinain', 'pinysn', 'pinyni', 'poinyin', 'pinybin', 'pnyin', 'prnyin', 'pinydn', 'pinymn', 'pinyins', 'picnyin', 'pinykn', 'pinyfin', 'pinyrn', 'pjnyin', 'pipyin', 'pincin', 'pinyiqn', 'pinein', 'pinyrin', 'dinyi

# Test

In [25]:
correct('yin')

'yin'

In [26]:
correct('yigna')

'yigna'

In [27]:
correct('yinn')

'ying'

In [28]:
def correct_sequence_pinyin(text_pinyin):
    return ' '.join(map(correct, text_pinyin.split()))

In [29]:
correct_sequence_pinyin('zhe sih yi ge ce sho')

'zhe shi yi ge ce shi'

In [30]:
correct_sequence_pinyin('wo xiang shagn qinng hua da xue')

'wo xiang shang qing hua da xue'

# 思考题-homework？    
#### 如何在不带空格的时候完成自动修整？--> 如何完成拼音的自动分割？   
###### 提示：使用第一节课提到的语言模型!

In [31]:
TOKEN = tokens(CHINESE_CHARATERS_COPYS)
words_count = Counter(TOKEN)

TOKEN_2_GRAM = [''.join(TOKEN[i:i+2]) for i in range(len(TOKEN[:-2]))]
words_count_2 = Counter(TOKEN_2_GRAM)

MAX_PLENGTH = max(map(len,TOKEN))

print(words_count.most_common(10))
print(TOKEN_2_GRAM[:10])

[('shi', 860634), ('de', 809887), ('n', 691114), ('yi', 682478), ('ji', 645276), ('guo', 430042), ('zhong', 409418), ('zhi', 398612), ('xin', 359619), ('li', 355441)]
['ciwai', 'waizi', 'ziben', 'benzhou', 'zhouyue', 'yueri', 'riqi', 'qichu', 'chuxiao', 'xiaomi']


In [32]:
def prob_1(word):
    if len(word) == 1:
        return 0
    if word in words_count:
        return (words_count[word] + 1) / (len(TOKEN) + len(words_count))
    else:
        return 1 / (len(TOKEN) + len(words_count))

def prob_2(w1, w2):
    combine = w1 + w2
    if combine in words_count_2:
        return (words_count_2[combine] + 1) / (len(TOKEN_2_GRAM) + len(words_count_2)) / prob_1(w1)
    else:
        return 1 / (len(TOKEN_2_GRAM) + len(words_count_2))
        
def get_probablity(sentence):
    words = sentence.split()
    sentence_pro = 1
    
    for i, word in enumerate(words[:-1]):
        next_ = words[i+1]       
        probability = prob_2(word, next_)
        sentence_pro *= probability
    
    return sentence_pro

In [33]:
def split_sentence(sentence):
    i = 0
    cands = []
    while i < len(sentence):       
        candicate = []
        for j in range(2, MAX_PLENGTH+2):
            word = sentence[i: i+j]
            word_c = correct(word)
            prod_word_c = prob_1(word_c)
            if prod_word_c > 1e-4:
                candicate.append((j, word, word_c, prob_1(word), prod_word_c))
        if candicate:
            candicate_tmp = sorted(candicate, key=lambda x: (-x[-2], -x[-1]))
            if len(candicate_tmp) > 3:
                candicate_tmp = candicate_tmp[:3]
            cands.append(list(set([x[2] for x in candicate_tmp])))
            i += candicate_tmp[0][0]
        else:
            i += 1
    return cands

In [34]:
def word_comb(l1, l2):
    res = []
    for i in l1:
        for j in l2:
            res.append((i + " " + j).strip())
    return res

In [35]:
def best_sentence(sentence):
    cands = split_sentence(sentence) 
    sences = cands[0]
    for w in cands[1:]:
        sences = word_comb(sences, w)

    sences_prob = []
    for s in sences:
        if len(s.split()) == len(cands):
            sences_prob.append((s, get_probablity(s)))
    
    return sorted(sences_prob, key=lambda x: -x[-1])[0][0]

In [36]:
best_sentence('woyaoshangqinghua')

'wo yao shang qing hua'

In [37]:
best_sentence('wo yaoshangqinghua')

'wo yao shang qing hua'

In [38]:
best_sentence('woyao shangqinghua')

'wo yao shang qing hua'

In [39]:
best_sentence('w yaoshangqinghua')

'wu yao shang qing hua'